# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Bañados  ->  E. Bañados  |  ['E. Bañados']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']
R. Zhang  ->  R. Zhang  |  ['R. Zhang']
J. Li  ->  J. Li  |  ['J. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


Arxiv has 83 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2410.03826


extracting tarball to tmp_2410.03826...

 done.


E. Bañados  ->  E. Bañados  |  ['E. Bañados']


...

E. Bañados  ->  E. Bañados  |  ['E. Bañados']


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure  sed_hists_delayedtau_v5.png
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=0.8\columnwidth]{ sed_hists_delayedtau_v5.png}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure  ewage_avgs_aug29_delayedtau.png
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=1.0\columnwidth]{ ewage_avgs_aug29_delayedtau.png}
  warnings.warn(LatexWarning(f"Could not fi

Found 92 bibliographic references in tmp_2410.03827/main.bbl.
Issues with the citations
syntax error in line 396: '=' expected
Retrieving document from  https://arxiv.org/e-print/2410.04467
Retrieving document from  https://arxiv.org/e-print/2410.04595


/tmp/ipykernel_2826/2822249172.py:52: LatexWarning: 2410.04467 did not run properly
HTTP Error 429: Too Many Requests
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


/tmp/ipykernel_2826/2822249172.py:52: LatexWarning: 2410.04595 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.03826-b31b1b.svg)](https://arxiv.org/abs/2410.03826) | **A Quasar-Anchored Protocluster at z=6.6 in the ASPIRE Survey: I. Properties of [OIII] Emitters in a 10 Mpc Overdensity Structure**  |
|| J. B. Champagne, et al. -- incl., <mark>E. Bañados</mark> |
|*Appeared on*| *2024-10-08*|
|*Comments*| *submitted to ApJ june 2024. 18 pages, 7 figures. companion paper II by Champagne+24b is also available on arXiv today*|
|**Abstract**|            ASPIRE (A SPectroscopic survey of bIased halos in the Reionization Era) is a quasar legacy survey primarily using JWST to target a sample of 25 $z>6$ quasars with NIRCam slitless spectroscopy and imaging. The first study in this series found evidence of a strong overdensity of galaxies around J0305-3150, a luminous quasar at $z=6.61$, within a single NIRCam pointing obtained in JWST Cycle 1. Here, we present the first results of a JWST Cycle 2 mosaic that covers 35 arcmin$^2$ with NIRCam imaging/WFSS of the same field to investigate the spatial extent of the putative protocluster. The F356W grism data targets [OIII]+H$\beta$ at $5.3<z<7$ and reveals a population of 124 line emitters down to a flux limit of 1.2$\times$10$^{-18}$ erg s$^{-1}$ cm$^{-2}$. Fifty-three of these galaxies lie at $6.5<z<6.8$ spanning 10 cMpc on the sky, corresponding to an overdensity within a 2500 cMpc$^3$ volume of 12.5 $\pm$ 2.6, anchored by the quasar. Comparing to the [OIII] luminosity function from the Emission line galaxies and Intergalactic Gas in the Epoch of Reionization (EIGER) project, we find a dearth of faint [OIII] emitters at log(L/erg s$^{-1}$) $<$ 42.3, which we suggest is consistent with either bursty star formation causing galaxies to scatter around the grism detection limit or modest suppression from quasar feedback. While we find a strong filamentary overdensity of [OIII] emitters consistent with a protocluster, we suggest that we could be insensitive to a population of older, more massive Lyman-break galaxies with weak nebular emission on scales $>10$ cMpc.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.03827-b31b1b.svg)](https://arxiv.org/abs/2410.03827) | **A Quasar-Anchored Protocluster at z=6.6 in the ASPIRE Survey: II. An Environmental Analysis of Galaxy Properties in an Overdense Structure**  |
|| J. B. Champagne, et al. -- incl., <mark>E. Bañados</mark> |
|*Appeared on*| *2024-10-08*|
|*Comments*| *accepted to ApJ. 24 pages, 13 figures. Paper I by Champagne+24a is also available on arxiv today*|
|**Abstract**|            We present paper II comprising a 35 arcmin$^2$ JWST/NIRCam imaging and wide-field slitless spectroscopy mosaic centered on J0305$-$3150, a luminous quasar at $z=6.61$. The F356W grism data reveals 124 [OIII]+H$\beta$ emitters at $5.3<z<7$, 53 of which constitute a protocluster spanning (10 cMpc)$^2$ across $6.5<z<6.8$. We find no evidence of any broad-line AGN in individual galaxies or stacking, reporting a median H$\beta$ FWHM of 585 $\pm$ 152 km s$^{-1}$; however, the mass-excitation diagram and ``little red dot" color and compactness criteria suggest that there are a few AGN candidates on the outskirts of the protocluster. We fit the spectral energy distributions (SEDs) of the [OIII] emitters with Prospector and Bagpipes, and find that none of the SED-derived properties (stellar mass, age, or star formation rate) correlates with proximity to the quasar. While there is no correlation between galaxy age and local galaxy density, we find modest correlations between local galaxy density with increasing stellar mass, decreasing 10-to-100 Myr star formation rate ratios and decreasing nebular line equivalent widths. We further find that the protocluster galaxies are consistent with being more massive, older, and hosting higher star formation rates than the field sample at the 3$\sigma$ level, distributed in a filamentary structure which supports inside-out formation of the protocluster. There is modest evidence that galaxy evolution proceeds differently as a function of the density of local environment within protoclusters during the epoch of reionization, and the central quasar has little effect on the galaxy properties of the surrounding structure.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.04467-b31b1b.svg)](https://arxiv.org/abs/2410.04467) | **Formation of Giant Radio Sources in Galaxy Clusters**  |
|| X. Duan, L. Wu, <mark>R. Zhang</mark>, <mark>J. Li</mark> |
|*Appeared on*| *2024-10-08*|
|*Comments*| *9 pages, 7 figures*|
|**Abstract**|            The number of observed giant radio sources (GRSs) has increased significantly in recent years, yet their formation mechanisms remain elusive. The discovery of giant radio galaxies within galaxy clusters has further intensified the ongoing this http URL utilize magnetohydrodynamic simulations to investigate the formation of GRSs in cluster this http URL avoid confounding the effects of power and total energy injection, we hold the energy of jet outbursts fixed and study the effect of power by varying the active duration of the jets. Furthermore, we examine the roles of magnetic, thermal, and kinetic energy components by adjusting their fractions in the jets. Additionally, we calculate radio emission for comparison with observations in the radio power-linear size diagram (P-D diagram). We find the 'lower power-larger bubble' effect: lower-power jets tend to produce larger radio sources with fixed total jet energy. Regarding different energy components, jets dominated by toroidal magnetic field energy generate larger radio sources than kinetic and thermal energy-dominated jets. Conversely, strong poloidal magnetic fields hinder radio lobe growth. When injecting $2.06 \times 10^{59}$ erg into a $10^{14}$ solar mass halo, only jets with powers of approximately $10^{-4}$ to $10^{-3}$ Eddington luminosity efficiently traverse the observational region in the P-D diagram. Our findings suggest that energetic, long-lasting (low-power), continuous jets endowed with significant toroidal magnetic fields facilitate the formation of GRSs in cluster environments. However, although the jets with significantly lower power can generate substantially larger radio sources, their faintness may render them unobservable.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error HTTP Error 429: Too Many Requests</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.04595-b31b1b.svg)](https://arxiv.org/abs/2410.04595) | **Atmospheres of Solar System Moons and Pluto**  |
|| <mark>X. Zhang</mark> |
|*Appeared on*| *2024-10-08*|
|*Comments*| *Chapter accepted for Encyclopedia of Atmospheric Sciences, Third Edition. 35 pages, 8 figures, and 2 tables*|
|**Abstract**|            The atmospheres within our Solar System can be categorized into four distinct climate regimes: "terrestrial", "Jovian", "condensable", and "exosphere". Beyond the three terrestrial planets (excluding Mercury) and the four giant planets, collisional atmospheres are also found on smaller celestial bodies such as Jupiter's moon Io, Saturn's moon Titan, Neptune's moon Triton, and Pluto. This article reviews the key characteristics of these atmospheres and the underlying physical and chemical processes that govern them. I focus on their thermal structures, chemical constituents, wind patterns, and the origins and losses of the atmospheres, and highlight the critical roles of surface ices and liquids, atmospheric hazes, and the space environments of their host planets in shaping these atmospheres. I dedicated this article to Prof. Zuo Xiao (1936-2024) at Peking University.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2410.03826/./hexbin_o3_mosaic_v5.png', 'tmp_2410.03826/./o3lf_mosaic_expmap_v1.png', 'tmp_2410.03826/./uvlf_v2.png', 'tmp_2410.03826/./full_f356w_map_v5.png']
copying  tmp_2410.03826/./hexbin_o3_mosaic_v5.png to _build/html/
copying  tmp_2410.03826/./o3lf_mosaic_expmap_v1.png to _build/html/
copying  tmp_2410.03826/./uvlf_v2.png to _build/html/
copying  tmp_2410.03826/./full_f356w_map_v5.png to _build/html/
exported in  _build/html/2410.03826.md
    + _build/html/tmp_2410.03826/./hexbin_o3_mosaic_v5.png
    + _build/html/tmp_2410.03826/./o3lf_mosaic_expmap_v1.png
    + _build/html/tmp_2410.03826/./uvlf_v2.png
    + _build/html/tmp_2410.03826/./full_f356w_map_v5.png
found figures ['', '']
file not found 
file not found 
exported in  _build/html/2410.03827.md
    + _build/html/
    + _build/html/


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\sfr}{M_{\odot} yr^{-1}}$
$\newcommand{\etal}{et al.}$
$\newcommand{\um}{\mum}$
$\newcommand{\lsun}{L_{\odot}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\mstar}{M_{\star}}$
$\newcommand{\lt}{<}$
$\newcommand{\gt}{>}$
$\newcommand{\Cii}{[C\textsc{ii}]}$
$\newcommand{\Oiii}{[O\textsc{iii}]}$
$\newcommand{\herschel}{\textit{Herschel}}$
$\newcommand{\lya}{Ly\alpha}$
$\newcommand{\halpha}{H\alpha}$
$\newcommand{\magphys}{\textsc{magphys}}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\pz}{P(z)}$
$\newcommand{\hband}{H_{160}}$
$\newcommand{\jband}{J_{125}}$
$\newcommand{\yband}{Y_{105}}$
$\newcommand{\iband}{i_{814}}$
$\newcommand{\vband}{V_{606}}$
$\newcommand{\jhband}{\textit{JH}_{140}}$
$\newcommand{\yjband}{\textit{YJ}_{110}}$
$\newcommand{\zband}{z_{850}}$
$\newcommand{\amp}{~}$
$\newcommand{\scubaii}{{\sc Scuba-2}}$
$\newcommand{\uJy}{\muJy}$
$\newcommand{\co}{\rm CO}$
$\newcommand{\alphaco}{\alpha_{\rm CO}}$
$\newcommand{\acounits}{M_{\odot}/(K km s^{-1} pc^{2})}$
$\newcommand{\ltsima}{\; \buildrel < \over \sim \;}$
$\newcommand{\simlt}{\lower.5ex\hbox{\ltsima}}$
$\newcommand{\gtsima}{\; \buildrel > \over \sim \;}$
$\newcommand{\simgt}{\lower.5ex\hbox{\gtsima}}$
$\newcommand{\ltsima}{\; \buildrel < \over \sim \;}$
$\newcommand{\simlt}{\lower.5ex\hbox{\ltsima}}$
$\newcommand{\gtsima}{\; \buildrel > \over \sim \;}$
$\newcommand{\simgt}{\lower.5ex\hbox{\gtsima}}$</div>



<div id="title">

# A Quasar-Anchored Protocluster at $z=6.6$ in the ASPIRE Survey: I. Properties of [OIII] Emitters in a 10 Mpc Overdensity Structure

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.03826-b31b1b.svg)](https://arxiv.org/abs/2410.03826)<mark>Appeared on: 2024-10-08</mark> -  _submitted to ApJ june 2024. 18 pages, 7 figures. companion paper II by Champagne+24b is also available on arXiv today_

</div>
<div id="authors">

Jaclyn~B.~Champagne, et al. -- incl., <mark>E. Bañados</mark>

</div>
<div id="abstract">

**Abstract:** ASPIRE (A SPectroscopic survey of bIased halos in the Reionization Era) is a quasar legacy survey primarily using _JWST_ to target a sample of 25 $z>6$ quasars with NIRCam slitless spectroscopy and imaging.The first study in this series found evidence of a strong overdensity of galaxies around J0305 $-$ 3150, a luminous quasar at $z=6.61$ , within a single NIRCam pointing obtained in JWST Cycle 1.Here, we present the first results of a JWST Cycle 2 mosaic that covers 35 arcmin $^2$ with NIRCam imaging/WFSS of the same field to investigate the spatial extent of the putative protocluster.The F356W grism data targets $\Oiii$ +H $\beta$ at $5.3<z<7$ and reveals a population of 124 line emitters down to a flux limit of 1.2 $\times$ 10 $^{-18}$ erg s $^{-1}$ cm $^{-2}$ .Fifty-three of these galaxies lie at $6.5<z<6.8$ spanning 10 cMpc on the sky, corresponding to an overdensity within a 2500 cMpc $^3$ volume of 12.5 $\pm$ 2.6, anchored by the quasar.Comparing to the $\Oiii$ luminosity function from the Emission line galaxies and Intergalactic Gas in the Epoch of Reionization (EIGER) project, we find a dearth of faint $\Oiii$ emitters at log(L/erg s $^{-1}$ ) $<$ 42.3, which we suggest is consistent with either bursty star formation causing galaxies to scatter around the grism detection limit or modest suppression from quasar feedback.While we find a strong filamentary overdensity of $\Oiii$ emitters consistent with a protocluster, we suggest that we could be insensitive to a population of older, more massive Lyman-break galaxies with weak nebular emission on scales $>10$ cMpc.

</div>

<div id="div_fig1">

<img src="tmp_2410.03826/./hexbin_o3_mosaic_v5.png" alt="Fig2" width="100%"/>

**Figure 2. -**  2D representation of spectroscopically confirmed $\Oiii$  emitters at the quasar redshift (circles). The quasar is marked as the large black $\times$; galaxies identified via other tracers are marked in pink symbols. The hexbin histogram (bottom colorbar) denotes the 2D overdensity of $\Oiii$  emitters with respect to the UV luminosity function at $z=6$ ([Finkelstein, Ryan and Papovich 2015]()) , showing that the overdensity is stronger in a filament pointing away from the quasar. The right colorbar denotes the redshift of individual galaxies, centered on the quasar redshift. The green outline depicts the footprint of the mosaic. (*fig:hexbin*)

</div>
<div id="div_fig2">

<img src="tmp_2410.03826/./o3lf_mosaic_expmap_v1.png" alt="Fig6.1" width="50%"/><img src="tmp_2410.03826/./uvlf_v2.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** _Left:_ Luminosity function of the $\Oiii$  emitters in our sample. The black points indicate members of the protocluster (empty point indicates the upper limit when including LBGs not detected in $\Oiii$); the blue and pink points indicate the overdensities at $z<6.5$. The green line is the blank field luminosity function from EIGER  ([Matthee, Mackenzie and Simcoe 2023]()) , and the filled gold points are the field galaxies in our sample that do not belong to the three overdensities. The grey shaded regions in both panels indicate where we are $<80$\% complete. The quasar overdensity is an order of magnitude above the blank field expectations but the shape is dissimilar, with a preference for fainter galaxies and a slight dearth of the brightest galaxies. _Right:_ The UV luminosity function within the quasar protocluster of our $\Oiii$  emitters and LBGs from [Champagne, Casey and Finkelstein (2023)](). M$_{UV}$ is calculated from the best-fit SEDs presented in Paper II. The redshift-parameterized UVLFs at $z=6.6$ and $z=4$ from [ and Finkelstein (2016)]() are shown: both the $\Oiii$  emitters and the LBGs are consistent with having undergone more rapid evolution than the field by several hundred Myr. (*fig:o3lf*)

</div>
<div id="div_fig3">

<img src="tmp_2410.03826/./full_f356w_map_v5.png" alt="Fig5" width="100%"/>

**Figure 5. -** 
    Full F356W six-pointing mosaic surrounding
    J0305$-$3150, with the full field of view outlined in black. The quasar is depicted at the center with a white circle. The quasar-anchored overdensity members are shown in pink, the overdensities at $z=5.4$ and $z=6.2$ are in blue and purple, and the field galaxies at $5.3<z<6.5$ are shown in yellow. The blue box shows the WFC3 footprint where galaxies have photometric coverage from _HST_. The configuration of the pointings is such that we have the greatest depth in the immediate surroundings of the quasar, so we are most sensitive to faint galaxies in the central regions. The filled points indicate sources already identified in the Cycle 1 data  ([Wang, Yang and Hennawi 2023]()) .
  Inset: Zoom-in to the central 30$\arcsec$  surrounding the quasar. Three companion $\Cii$  emitters  ([Venemans, Walter and Neeleman 2020]()) , 1 dust continuum emitter  ([Champagne, Decarli and Casey 2018]()) , 1 LAE  ([Farina, Venemans and Decarli 2017]()) , and 2 $\Oiii$  emitters (this work; one of them is the LAE) are visible within a few hundred ckpc from the quasar. (*fig:f356w*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.03826"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\sfr}{M_{\odot} yr^{-1}}$
$\newcommand{\etal}{et al.}$
$\newcommand{\um}{\mum}$
$\newcommand{\lsun}{L_{\odot}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\mstar}{M_{\star}}$
$\newcommand{\lt}{<}$
$\newcommand{\gt}{>}$
$\newcommand{\Cii}{[C\textsc{ii}]}$
$\newcommand{\Oiii}{[O\textsc{iii}]}$
$\newcommand{\herschel}{\textit{Herschel}}$
$\newcommand{\lya}{Ly\alpha}$
$\newcommand{\halpha}{H\alpha}$
$\newcommand{\magphys}{\textsc{magphys}}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\pz}{P(z)}$
$\newcommand{\hband}{H_{160}}$
$\newcommand{\jband}{J_{125}}$
$\newcommand{\yband}{Y_{105}}$
$\newcommand{\iband}{i_{814}}$
$\newcommand{\vband}{V_{606}}$
$\newcommand{\jhband}{\textit{JH}_{140}}$
$\newcommand{\yjband}{\textit{YJ}_{110}}$
$\newcommand{\zband}{z_{850}}$
$\newcommand{\amp}{~}$
$\newcommand{\scubaii}{{\sc Scuba-2}}$
$\newcommand{\uJy}{\muJy}$
$\newcommand{\co}{\rm CO}$
$\newcommand{\alphaco}{\alpha_{\rm CO}}$
$\newcommand{\acounits}{M_{\odot}/(K km s^{-1} pc^{2})}$
$\newcommand{\ltsima}{\; \buildrel < \over \sim \;}$
$\newcommand{\simlt}{\lower.5ex\hbox{\ltsima}}$
$\newcommand{\gtsima}{\; \buildrel > \over \sim \;}$
$\newcommand{\simgt}{\lower.5ex\hbox{\gtsima}}$
$\newcommand{\ltsima}{\; \buildrel < \over \sim \;}$
$\newcommand{\simlt}{\lower.5ex\hbox{\ltsima}}$
$\newcommand{\gtsima}{\; \buildrel > \over \sim \;}$
$\newcommand{\simgt}{\lower.5ex\hbox{\gtsima}}$</div>



<div id="title">

# A Quasar-Anchored Protocluster at $z=6.6$ in the ASPIRE Survey: II. An Environmental Analysis of Galaxy Properties in an Overdense Structure

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.03827-b31b1b.svg)](https://arxiv.org/abs/2410.03827)<mark>Appeared on: 2024-10-08</mark> -  _accepted to ApJ. 24 pages, 13 figures. Paper I by Champagne+24a is also available on arxiv today_

</div>
<div id="authors">

Jaclyn~B.~Champagne, et al. -- incl., <mark>E. Bañados</mark>

</div>
<div id="abstract">

**Abstract:** We present paper II comprising a 35 arcmin $^2$ _JWST_ /NIRCam imaging and wide-field slitless spectroscopy mosaic centered on J0305 $-$ 3150, a luminous quasar at $z=6.61$ .The F356W grism data reveals 124 $\Oiii$ +H $\beta$ emitters at $5.3<z<7$ , 53 of which constitute a protocluster spanning (10 cMpc) $^2$ across $6.5<z<6.8$ .We find no evidence of any broad-line AGN in individual galaxies or stacking, reporting a median H $\beta$ FWHM of 585 $\pm$ 152 km s $^{-1}$ ; however, the mass-excitation diagram and "little red dot" color and compactness criteria suggest that there are a few AGN candidates on the outskirts of the protocluster.We fit the spectral energy distributions (SEDs) of the $\Oiii$ emitters with \texttt{Prospector} and \texttt{Bagpipes} , and find thatnone of the SED-derived properties (stellar mass, age, or star formation rate) correlates with proximity to the quasar.While there is no correlation between galaxy age and local galaxy density, we find modest correlations between local galaxy density with increasing stellar mass, decreasing 10-to-100 Myr star formation rate ratios and decreasing nebular line equivalent widths.We further find that the protocluster galaxies are consistent with being more massive, older, and hosting higher star formation rates than the field sample at the 3 $\sigma$ level, distributed in a filamentary structure which supports inside-out formation of the protocluster.There is modest evidence that galaxy evolution proceeds differently as a function of the density of local environment within protoclusters during the epoch of reionization, and the central quasar has little effect on the galaxy properties of the surrounding structure.

</div>

<div id="div_fig1">

<img src="" alt="Fig4.1" width="50%"/><img src="" alt="Fig4.2" width="50%"/>

**Figure 4. -**  Example cutouts and spectrum for ID7025, a member of the quasar protocluster. _Top:_ 2$\arcsec$$\times$2$\arcsec$ cutouts in each filter from Subaru, _HST_ and _JWST_, where the red circle denotes the circular aperture used for photometry. _Middle:_ 2D co-added grism spectrum highlighting the detection of three emission lines. _Bottom:_ Optimally extracted 1D grism spectrum (green) and error (pink) with the locations of H$\beta$, $\Oiii$$\lambda$4959 and $\Oiii$$\lambda$5007 labeled with thick blue lines. (*fig:example*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.03827"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

210  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
